# Librerias

In [3]:
import pandas as pd
import numpy as np
import requests
import json
import time
from dotenv import dotenv_values

pd.set_option('max_colwidth', None)

### Nearby Search

In [5]:
config = dotenv_values(".env")

In [ ]:
lugares = []

norte = 40.485657
sur = 40.389104
este = -3.659151
oeste = -3.749617
step = 0.01


for lat in np.arange(sur, norte, step):
    for lon in np.arange(oeste, este, step):
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        params = {'location': f'{lat},{lon}',
          'radius': 1000,
          'type': 'restaurant',
          'key':config['access_key']}

        for page in range(3):
            res = requests.get(url, params=params)
            #print(res)
            data = res.json()
            lugares += data.get('results', [])
            next_page_token = data.get('next_page_token')
            if next_page_token is not None:
                time.sleep(2)
                params = {'pagetoken': next_page_token, 'key': config['access_key']}
            else:
                break

lugares_unicos = {lugar['place_id']: lugar for lugar in lugares}

In [119]:
with open('../data/raw/lugares_madrid_raw.csv', 'w') as file:
     file.write(json.dumps(lugares_unicos))

In [122]:
def json_to_dataframe(data_entrada):
    
    places = {'nombre': [], 'id': [], 'lat':[], 'lon':[]}

    for key, val in data_entrada.items():
        nombre = data_entrada[key]['name']
        lat = data_entrada[key]['geometry']['location']['lat']
        lon = data_entrada[key]['geometry']['location']['lng']
        id = key
        
        places['nombre'].append(nombre)
        places['id'].append(id)
        places['lat'].append(lat)
        places['lon'].append(lon)

    df = pd.DataFrame(places)
    return df

lugaresdf = json_to_dataframe(lugares_unicos)

In [123]:
lugaresdf.shape

(2628, 4)

In [124]:
lugaresdf.to_csv('../data/raw/lugares_madrid.csv', index=False)

### Places Detail

In [ ]:
fields = ",".join(["place_id", "formatted_address",
    "delivery", "dine_in", "editorial_summary",
    "price_level", "rating", "reservable", "reviews",
    "serves_beer", "serves_breakfast", "serves_brunch",
    "serves_dinner", "serves_lunch", "serves_vegetarian_food",
    "serves_wine", "takeout", "user_ratings_total"
])

detalle_sitios = []

for i in lugaresdf['id']:
    url = 'https://maps.googleapis.com/maps/api/place/details/json'
    
    params = {'place_id': i,
            'fields': fields,
            'key':config['access_key']}

    details = requests.get(url, params=params)
    resultado = details.json().get('result', {})
    detalle_sitios.append(resultado)

In [126]:
with open('../data/raw/detalle_sitios_raw.csv', 'w') as file:
     file.write(json.dumps(detalle_sitios))

In [161]:
df_detalle = {  'place_id':[],
                'dine_in':[],
                'summary':[],
                'address':[],
                'price_level':[],
                'rating':[],
                'reservable':[],
                'serves_beer':[],
                'serves_breakfast':[],
                'serves_brunch':[],
                'serves_dinner':[],
                'serves_lunch':[],
                'serves_vegetarian_food':[],
                'serves_wine':[],
                'takeout':[],
                'user_ratings_total':[],
                'delivery':[]
                }

for i in detalle_sitios:
    df_detalle['place_id'].append(i.get('place_id'))
    df_detalle['dine_in'].append(i.get('dine_in', np.nan))
    df_detalle['address'].append(i.get('formatted_address', ''))
    df_detalle['rating'].append(i.get('rating', np.nan))
    df_detalle['user_ratings_total'].append(i.get('user_ratings_total', np.nan))
    df_detalle['delivery'].append(i.get('delivery', np.nan))
    df_detalle['summary'].append(i.get('editorial_summary', {}).get('overview', ''))
    df_detalle['price_level'].append(i.get('price_level', np.nan))
    df_detalle['reservable'].append(i.get('reservable', np.nan))
    df_detalle['serves_beer'].append(i.get('serves_beer', np.nan))
    df_detalle['serves_breakfast'].append(i.get('serves_breakfast', np.nan))
    df_detalle['serves_brunch'].append(i.get('serves_brunch', np.nan))
    df_detalle['serves_dinner'].append(i.get('serves_dinner', np.nan))
    df_detalle['serves_lunch'].append(i.get('serves_lunch', np.nan))
    df_detalle['serves_vegetarian_food'].append(i.get('serves_vegetarian_food', np.nan))
    df_detalle['serves_wine'].append(i.get('serves_wine', np.nan))
    df_detalle['takeout'].append(i.get('takeout', np.nan))

In [164]:
detalle_sitios_df = pd.DataFrame(df_detalle)

In [165]:
detalle_sitios_df.to_csv('../data/raw/detalle_sitios.csv', index=False)

### Open street map

In [53]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json][timeout:25];
area["name"="Madrid"]->.searchArea;
(
  node["amenity"="restaurant"](area.searchArea);
  way["amenity"="restaurant"](area.searchArea);
  relation["amenity"="restaurant"](area.searchArea);
);
out center;
"""

response = requests.get(overpass_url, params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


In [54]:
street = {'nombre':[],
          'postal':[],
          'calle':[],
          'cocina':[],
          'lat':[],
          'lon':[],
          'wheelchair':[]}

for i in data['elements']:
    street['nombre'].append(i['tags'].get('name', ''))
    street['postal'].append(i['tags'].get('addr:postcode', ''))
    street['calle'].append(i['tags'].get('addr:street', ''))
    street['cocina'].append(i['tags'].get('cuisine', ''))
    street['lat'].append(i.get('lat', ''))
    street['lon'].append(i.get('lon', ''))
    street['wheelchair'].append(i['tags'].get('wheelchair', ''))

street_df = pd.DataFrame(street)

In [55]:
street_df.shape

(4644, 7)

In [56]:
street_df.to_csv('../data/raw/sitios_streetmap.csv', index=False)

### Ayuntamiento Madrid : https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=71359583a773a510VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [124]:
df_barrios = pd.read_csv('../data/raw/panel_indicadores_distritos_barrios.csv', sep=';', index_col='Orden')
df_barrios = df_barrios[df_barrios['Periodo panel']==2024]

In [125]:
df_barrios['indicador_completo'] = df_barrios['indicador_completo'].str.strip()

In [126]:
df_barrios = df_barrios[(df_barrios['indicador_completo'].isin(
    ['Población densidad (hab./Ha.)', 'Número Habitantes',
        'Total hogares',
        'Renta disponible media por persona',
        'Calidad de vida actual en su barrio',
        'Satisfacción de vivir en su barrio',
        'Percepción de seguridad en el barrio por la noche',
        'Percepción de seguridad en el barrio por el día']))]


In [127]:
df_barrios = df_barrios[(df_barrios['cod_distrito'].notna())&(df_barrios['cod_barrio'].notna())]

df_barrios.drop(['Periodo panel', 'ciudad', 
                 'fecha_indicador', 'fuente_indicador', 
                 'categoría_1', 'categoría_2', 'indicador_nivel1',
                 'indicador_nivel2', 'indicador_nivel3'], inplace=True, axis=1)

In [128]:
df_barrios.to_csv('../data/raw/kpi_barrios_madrid.csv', index=False)